### 📖 Where We Are

**So far**, our journey has taken us through various data formats:
1.  **Notebooks 1-3**: Covered unstructured files like `.txt`, `.pdf`, and `.docx`.
2.  **Notebooks 4-5**: Handled structured and semi-structured file formats like CSV, Excel, and JSON.

**In this notebook**, we take a significant step forward by learning to parse data directly from a **live SQL database**. This is a different paradigm from reading static files. Our focus will be on converting relational tables, their schemas, and the relationships between them into a text-based format that a Large Language Model can effectively use for Retrieval-Augmented Generation.

### 1. SQL Databases

Interfacing with SQL databases for RAG involves more than just dumping table rows into documents. To effectively answer questions, the LLM needs context about the **database schema**—what tables exist, what their columns are, and how they relate to each other.

**Analogy**: Think of a SQL database as a perfectly organized library with many different collections of books (tables). 
- A **naive approach** would be to just copy the text from every page of every book. A researcher (the LLM) would be overwhelmed and have no idea how the books are related. 
- A **smart approach** is to provide the researcher with the library's **master catalog** (the database schema). This catalog describes each collection, its purpose, and how it cross-references other collections. You might also provide summaries of key information and relationships.

Our goal is to create these 'catalog entries' and 'summaries' as `Document` objects. We'll start by setting up a simple SQLite database.

In [1]:
# `sqlite3` is Python's built-in library for interacting with SQLite databases.
import sqlite3
import os

os.makedirs("data/databases", exist_ok=True)

In [2]:
# Connect to the SQLite database. This will create the file if it doesn't exist.
conn = sqlite3.connect('data/databases/company.db')
# A cursor object is used to execute SQL commands.
cursor = conn.cursor()

In [3]:
# Create an 'employees' table with a defined schema if it doesn't already exist.
# This schema defines the columns and their data types.
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
                 (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')

In [4]:
# Create a 'projects' table. Note the 'lead_id' column, which will relate to the 'id' in the employees table.
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
                 (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [5]:
# Define sample data to insert into our tables.
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]

In [6]:
# Use `executemany` to efficiently insert multiple rows into the tables.
# `INSERT OR REPLACE` will update a row if the primary key already exists, preventing duplicates.
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [7]:
# Commit the changes to save them to the database file.
conn.commit()
# Always close the connection when you're done.
conn.close()

## 2. Database Content Extraction

In [8]:
# SQLDatabase: A LangChain utility that acts as a wrapper around any SQL database.
from langchain_community.utilities import SQLDatabase
# SQLDatabaseLoader: A loader that can execute a query and load the results.
from langchain_community.document_loaders import SQLDatabaseLoader

### Method 1: `SQLDatabase` Utility (Schema-Focused)

The `SQLDatabase` utility is a high-level wrapper that connects to your database and provides convenient methods for getting schema information. This is extremely useful for **Text-to-SQL** applications, where you need to give an LLM the database schema so it can write its own queries. For standard RAG, this information can be ingested as context.

In [9]:
# Connect to the database using a URI (Uniform Resource Identifier).
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

# Get a list of table names in the database.
print(f"Tables: {db.get_usable_table_names()}")

# get_table_info() is a powerful method that returns the SQL DDL (CREATE TABLE statements)
# for each table, along with a few sample rows. This is excellent context for an LLM.
print(f"\nTable DDL:")
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


### Method 2: `SQLDatabaseLoader` (Query-Focused)

This loader connects to the database, runs a specific query you provide, and treats **each row** of the result as a separate `Document`. This is the most direct way to ingest specific data subsets from your database.

In [10]:
print("2️⃣ SQLDatabaseLoader")
# Define the query we want to run.
query = "SELECT id, name, role FROM employees WHERE department = 'Engineering'"

# Initialize the loader with the db object and the query.
loader = SQLDatabaseLoader(db=db, query=query)

# Load the data. Each row becomes a Document.
docs_from_loader = loader.load()

print(f"Loaded {len(docs_from_loader)} documents from the query.")
print("First document's content:")
# The page_content is a string representation of the row's dictionary.
print(docs_from_loader[0].page_content)
print("\nFirst document's metadata:")
print(docs_from_loader[0].metadata)

2️⃣ SQLDatabaseLoader
Loaded 2 documents from the query.
First document's content:
id: 1
name: John Doe
role: Senior Developer

First document's metadata:
{}


### Method 3: Custom SQL to Document Conversion (Content-Focused)

For many RAG use cases, providing only the schema or raw rows isn't enough. We want to create rich, descriptive documents from the database content itself. A custom function gives us full control to:
1.  Create a summary `Document` for each table.
2.  Create `Document` objects that describe the **relationships** between tables by using SQL `JOIN` queries. This pre-computes valuable insights that an LLM would otherwise have to figure out on its own.

In [11]:
from typing import List
from langchain_core.documents import Document

print("\n3️⃣ Custom SQL Processing")

def sql_to_documents(db_path: str) -> List[Document]:
    """Connects to a SQL DB and creates structured Documents for tables and their relationships."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []
    
    # --- Strategy 1: Create a summary document for each table ---
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        # Get column names for the table.
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = [col[1] for col in cursor.fetchall()]
        # Get all data from the table.
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        
        # Format the table summary into a readable string.
        table_content = f"Table Name: {table_name}\nColumns: {', '.join(columns)}\nTotal Records: {len(rows)}\n\nSample Records:\n"
        for row in rows[:3]: # Add first 3 records as samples
            record = dict(zip(columns, row))
            table_content += f"{record}\n"
        
        doc = Document(page_content=table_content, metadata={'source': db_path, 'table_name': table_name})
        documents.append(doc)

    # --- Strategy 2: Create a document describing relationships using a JOIN ---
    # This query finds which employee leads which project.
    query = """
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e JOIN projects p ON e.id = p.lead_id
    """
    cursor.execute(query)
    relationships = cursor.fetchall()
    
    # Format the relationships into a natural language summary.
    rel_content = "Employee-Project Leadership Summary:\n"
    for rel in relationships:
        rel_content += f"- {rel[0]} (Role: {rel[1]}) is the lead for the '{rel[2]}' project, which is currently {rel[3]}.\n"
    
    rel_doc = Document(page_content=rel_content, metadata={'source': db_path, 'data_type': 'relationships'})
    documents.append(rel_doc)
    
    conn.close()
    return documents


3️⃣ Custom SQL Processing


In [12]:
# Execute the custom function and see the resulting documents.
sql_docs = sql_to_documents("data/databases/company.db")
sql_docs

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees'}, page_content="Table Name: employees\nColumns: id, name, role, department, salary\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects'}, page_content="Table Name: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.0, 'lead_id': 1}\n{'id': 2, 'name': 'Data Pipeline', 'status': 'Completed', 'budget': 80000.0, 'lead_id': 2}\n{'id': 3, 'name': 'Customer Portal', 'status': 'Planning', 'budget': 200000.0, '

### 📊 SQL Processing Strategy Comparison

| Strategy | Output | Key Feature | Best For |
| :--- | :--- | :---: | :--- |
| **`SQLDatabase` Utility** | Schema DDL & sample rows | Provides the database's blueprint. | **Text-to-SQL Agents**. Gives an LLM the structural information it needs to write its own `SELECT` queries. |
| **`SQLDatabaseLoader`** | One `Document` per row from a query | Simple, direct data extraction. | **Quickly ingesting specific data subsets**. Useful for loading raw query results (e.g., all product descriptions) for embedding. |
| **Custom Function** | Formatted text summaries of tables and relationships. | Pre-computes and summarizes key insights. | **Standard RAG**. The most powerful option. Creates high-quality documents (like table summaries and relationship descriptions) for answering complex questions. |

### 🔑 Key Takeaways

* **Schema is Context**: When working with SQL databases for RAG, providing the schema (tables, columns) is as important as the data itself. The LLM needs this blueprint to understand the data's structure.
* **LangChain Utilities for Different Goals**: `SQLDatabase` is for getting schema, while `SQLDatabaseLoader` is for getting raw data from a query. They serve different but complementary purposes.
* **Custom Functions for Content**: For a sophisticated RAG workflow, a custom function that connects to the database is more powerful. It allows you to create bespoke `Document` objects that summarize tables in a readable format.
* **Unlock Insights with JOINs**: The most valuable documents you can create from a relational database are often those that describe the **relationships** between tables. Writing a SQL `JOIN` query and formatting the results into a natural language summary provides powerful, pre-computed context for the LLM.